## Questão 01

In [15]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [19]:
response_vectors = np.array([
    [1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, -1],
    [-1, -1, -1, -1, -1, -1, -1, 1]
], dtype=np.float32)

data = []

n = 1000
for i in range(n):
    instance = np.array([random.randint(0,1), random.randint(0,1), random.randint(0,1)])
    label = [instance[0]*4 + instance[1]*2 + instance[2]]
    noise = np.array([random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1)])

    data.append(np.concatenate((instance + noise, label)))


data = np.array(data, dtype=np.float32)

train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader

In [22]:
class CubeVertexesDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        X = self.data[idx, :3]
        y = self.data[idx, 3]
        return X, y
    
train_tensor = torch.tensor(train)
val_tensor = torch.tensor(val)
test_tensor = torch.tensor(test)

train_dataloader = DataLoader(CubeVertexesDataset(train_tensor), batch_size=1024)
val_dataloader = DataLoader(CubeVertexesDataset(val_tensor), batch_size=1024)
test_dataloader = DataLoader(CubeVertexesDataset(test_tensor), batch_size=1024)

In [24]:
import torch.nn as nn
import torch.nn.functional as F

In [27]:
class Rosemblatt(nn.Module):
    def __init__(self):
        super(Rosemblatt, self).__init__()
        self.perceptron = nn.Linear(3, 8, dtype=torch.float64)

    def forward(self, x):
        x = self.perceptron(x)
        return F.softmax(x, dim=1, dtype=torch.float64)

In [28]:
device = ("cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu")

model = Rosemblatt().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

In [29]:
def train(dataloader, model, loss_fn, optimizer):
    
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        y = y.type(torch.LongTensor)

        # O vetor de features e o seu label são passados para o device
        X, y = X.to(device), y.to(device)

        # As features são passadas pro modelo, que retorna um valor entre -1 e 1,
        # devido à função de ativação softmax, o que vai ser o resultado da classificação
        out = model(X)

        # O erro é calculado
        loss = loss_fn(out, y)

        # Backpropagation e atualização dos pesos.
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

        # O erro é exibido a cada 1000.
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [21]:
# import plotly.graph_objects as go
# fig = go.Figure(data=[go.Scatter3d(x=X_test[:,0], y=X_test[:,1], z=X_test[:,2], mode='markers', marker=dict(size=3, color=y_pred))])
# fig.show()